In [0]:
# Define widgets with default values
dbutils.widgets.text("quote_id", "R9999")
dbutils.widgets.text("catalog", "lrcatalog")
dbutils.widgets.text("schema", "agentic_underwriting")

#%pip install -U databricks-agents databricks-openai databricks-langchain mlflow
#dbutils.library.restartPython()


In [0]:
# Get widget values
quote_id = dbutils.widgets.get("quote_id")
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

### Here we start with tool definitions for our agent.


### 🔍 Function: `make_get_quote_details`

This function factory returns a callable that retrieves details of a specific motor insurance quote from the `quotes` table in the specified catalog and schema.

- **Input:** `quote_id` (case-insensitive, trimmed string)
- **Output:** Markdown-formatted table with quote details, or an error message if not found
- **Use case:** Can be plugged into LangChain agents or UI apps to display quote details based on user input

In [0]:
def make_get_quote_details(catalog, schema):
    def _fn(quote_id: str) -> str:
        quote_id_clean = quote_id.strip()
        df = spark.sql(f"""
            SELECT * FROM {catalog}.{schema}.quotes
            WHERE LOWER(quote_id) = LOWER('{quote_id_clean}')
        """).toPandas()

        if df.empty:
            return f"🚫 Quote not found for quote_id: {quote_id_clean}"
        
        markdown = df.to_markdown(index=False)
        return f"✅ Quote found:\n\n{markdown}"
    return _fn


### 🔍 Function: `make_validate_claims`

This function factory creates a callable that validates disclosed claims for a customer based on their full name and postcode.

- **Input:** A string in the format `'First Last, POSTCODE'`
- **Output:** A Markdown-formatted table of matching claim validation records, or an error message
- **Use case:** Useful in LangChain agents or UI flows to check if the customer’s disclosed claims match stored records before underwriting

In [0]:
def make_validate_claims(catalog, schema):
    def _fn(full_name_and_postcode: str) -> str:
        def escape_sql(value: str) -> str:
            return value.replace("'", "''")

        try:
            name_part, postcode = full_name_and_postcode.rsplit(",", 1)
            first_name, last_name = name_part.strip().split(" ", 1)
        except ValueError:
            return "Please provide input as 'First Last, POSTCODE'."

        first_name = escape_sql(first_name.strip())
        last_name = escape_sql(last_name.strip())
        postcode = escape_sql(postcode.strip())

        df = spark.sql(f"""
            SELECT * FROM {catalog}.{schema}.claims_disclosure_validation
            WHERE first_name = '{first_name}' AND last_name = '{last_name}' AND postcode = '{postcode}'
        """).toPandas()

        if df.empty:
            return "No claims record found for this name and postcode."
        return df.to_markdown(index=False)
    return _fn


### 🔍 Function: `make_get_call_transcript`

This function factory returns a callable that retrieves the call transcript associated with a specific motor insurance quote from the `sales_call_transcripts` table.

- **Input:** `quote_id` (string)
- **Output:** Raw text of the call transcript, or a message if no transcript is found
- **Use case:** Can be used in LangChain agents or apps to provide customer interaction history for underwriting or sales analysis

In [0]:
def make_get_call_transcript(catalog, schema):
    def _fn(quote_id: str) -> str:
        df = spark.sql(f"""
            SELECT call_transcript
            FROM {catalog}.{schema}.sales_call_transcripts
            WHERE quote_id = '{quote_id}'
        """).toPandas()

        if df.empty:
            return f"No call transcript available for quote ID: {quote_id}"

        return df.iloc[0]["call_transcript"]
    return _fn


### 🔍 Function: `score_quote_tool`

This function scores a motor insurance quote using a basic risk model based on age, vehicle type, number of claims, and no claims discount (NCD).

- **Input:** A dictionary or JSON string with keys: `age`, `vehicle_type`, `ncd_declared`, and `claims_declared`
- **Output:** A dictionary containing the calculated `price`, the `model_name`, and optionally an `error` message
- **Use case:** Can be used in LangChain agents or quote evaluation tools to simulate a pricing model based on risk factors

In [0]:
import json

def score_quote_tool(inputs) -> dict:
    """
    Accepts either a dictionary or a JSON string with keys:
    age, vehicle_type, ncd_declared, claims_declared, storage_declared.
    Returns a dict with price and model name.
    """

    if isinstance(inputs, str):
        try:
            inputs = json.loads(inputs)
        except json.JSONDecodeError:
            return {
                "price": None,
                "model_name": "standard_risk_model",
                "error": "Invalid input: cannot parse JSON input string."
            }

    def to_number_or_empty(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return ''

    # Extract values safely
    age = to_number_or_empty(inputs.get("age"))
    vehicle_type = (inputs.get("vehicle_type") or '').strip()
    ncd_declared = to_number_or_empty(inputs.get("ncd_declared"))
    claims_declared = to_number_or_empty(inputs.get("claims_declared"))
    storage_declared = (inputs.get("storage_declared") or '').strip().lower()

    # Validate required numeric fields
    if '' in (age, ncd_declared, claims_declared):
        return {
            "price": None,
            "model_name": "standard_risk_model",
            "error": "Invalid input: age, ncd_declared, and claims_declared must be numbers."
        }

    # Pricing logic
    base_price = 500
    claim_adj = claims_declared * 100

    vehicle_mult = {
        "SUV": 1.2,
        "Hatchback": 1.0,
        "Sports": 1.5
    }.get(vehicle_type, 1.1)

    if age < 25:
        age_adj = 200
    elif age < 35:
        age_adj = 100
    else:
        age_adj = 0

    storage_adj = {
        "garage": -50,
        "driveway": 0,
        "street": 50
    }.get(storage_declared, 25)  # fallback adjustment

    ncd_adj = ncd_declared * 30

    quote_value = round((base_price + claim_adj + age_adj + storage_adj - ncd_adj) * vehicle_mult, 2)

    return {
        "price": quote_value,
        "model_name": "standard_risk_model"
    }


### 🏡 Function: `validate_property_attributes`

This function retrieves property-level risk data for a given postcode, such as garage and driveway availability and overall property risk level.

- **Input:** A UK postcode as a plain string (e.g., `'CR3 6JE'`)
- **Output:** A markdown table of the matching row from the `property_attributes` table, or a message if no match is found
- **Use case:** Used in LangChain agents or underwriting pipelines to enrich or validate property context based on location

In [0]:
def make_validate_property_attributes(catalog, schema):
    def _fn(postcode: str) -> str:
        def escape_sql(value: str) -> str:
            return value.replace("'", "''")

        postcode = escape_sql(postcode.strip())

        df = spark.sql(f"""
            SELECT * FROM {catalog}.{schema}.property_attributes
            WHERE postcode = '{postcode}'
        """).toPandas()

        if df.empty:
            return f"No property record found for postcode '{postcode}'."
        return df.to_markdown(index=False)
    
    return _fn

In [0]:
# def make_get_driver_risk(catalog, schema):
#     def _fn(age_str: str) -> str:
#         def escape_sql(v): return v.replace("'", "''")
#         try:
#             age = int(age_str)
#         except ValueError:
#             return "Please provide a valid age as a number."

#         df = spark.sql(f"""
#             SELECT * FROM {catalog}.{schema}.driver_risk_profile
#             WHERE age = {age}
#         """).toPandas()

#         if df.empty:
#             return f"No risk profile found for age {age}."
#         return df.to_markdown(index=False)
#     return _fn


### 🧰 Tool Definitions for LangChain Agent

This section defines the set of tools available to the LangChain agent, each wrapping a callable function with a clear name and description.

- **Tools included:**
  - `Get Quote Details`: Returns declared customer and quote info by quote ID
  - `Validate Claims`: Returns verified NCD and claims using name and postcode
  - `Get Call Transcript`: Retrieves the sales call transcript for a quote ID
  - `Score Quote`: Calculates a new quote price using risk-based inputs

- **Use case:** These tools enable the agent to retrieve and cross-check underwriting data, simulate new pricing, and analyze sales interactions in real-time

In [0]:
from langchain.tools import Tool

tools = [
    Tool.from_function(make_get_quote_details(catalog, schema), name="Get Quote Details",description="Returns the declared quote data including first name, last name, postcode, age, vehicle type, declared claims and declared NCD. Input Use this to compare with validated data. Input just the quote ID value."),
    Tool.from_function(make_validate_claims(catalog, schema), name="Validate Claims", description="Returns the verified amount of claims and NCD for the given person. Compare these with what the customer declared in the quote using first name, last name and postcode.  Input two values: name and postcode. "),
    Tool.from_function(make_get_call_transcript(catalog, schema), name="Get Call Transcript", description="Returns the sales call transcript for a given quote ID if available. Analyse transcript data and compare with what the quote and validation data claim. Look for any potential errors of sales representative, for example mixed values in fields."),
    Tool.from_function(score_quote_tool, name="Score Quote", description="Returns a price and model version used to generate it. Input values in this format {{\"age\": 32, \"vehicle_type\": \"SUV\", \"ncd_declared\": 6, \"claims_declared\": 3, \"storage_declared\": \"garage\"}}. Can be used to create a new price."),
    Tool.from_function(make_validate_property_attributes(catalog, schema), name="Validate Property Attributes", description=("Returns property attributes for a given postcode, such as has_garage, has_driveway, and property_risk_level. Input must be a UK postcode in the format: 'CR3 6JE'. Useful for enriching underwriting decisions with contextual property risk data.")
    )
    #Tool.from_function(make_get_driver_risk(catalog, schema), name="Get Driver Risk", description="Get the risk segment for a postcode.")
]


### 🧠 Agent Initialization

This cell initializes a LangChain agent using a Databricks-hosted LLM (`meta-llama-3-1-70b-instruct`) and a set of predefined tools.

- **LLM:** `ChatDatabricks` connected to a specified endpoint
- **Agent type:** `ZERO_SHOT_REACT_DESCRIPTION` — allows reasoning over tool descriptions without examples
- **Settings:**
  - `verbose=True` for step-by-step logging
  - `handle_parsing_errors=True` to gracefully manage response formatting issues

- **Use case:** Powers a dynamic, tool-using agent capable of answering insurance-related questions or performing quote evaluations

In [0]:
from databricks_langchain import ChatDatabricks
from langchain.agents import initialize_agent, AgentType

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)


### 📝 Agent Execution Workflow

This cell runs a multi-step underwriting evaluation process through the LangChain agent using the specified quote ID.

- **Goal:** Assist an underwriter in checking quote accuracy and flagging issues before approval
- **Steps:**
  1. **Get Quote Details** – fetch declared quote information
  2. **Validate Claims** – check against verified claims/NCD data
  3. **Get Call Transcript** – verify customer disclosures against quote data
  4. **Identify Errors** – if call center input error is suspected, rescore using `Score Quote`
  5. **Summarize** – return a comprehensive decision message

- **Use case:** Enables end-to-end decision support with data validation, transcript analysis, and dynamic pricing review

In [0]:
agent_output = agent.run(f"""
You are an underwriter for motor insurance policies. You will check a quote and decide whether it should be approved or reviewed.

Step 1: Use the 'Get Quote Details' tool with quote ID {quote_id}. If the quote is not found, respond with a message and stop the process. Otherwise, get the quote details and Return the message.

Step 2: Use the 'Validate Claims'. If the customer does not exist in validation data, flag potential fraud, respond with correct message and stop this process. Otherwise check the number of claims and ncd, compare to the quote and decide if it should be approved or reviewed. Return the relevant message.

Step 3: Use the 'Get Call Transcript' tool to retrieve the sales call for this quote. Check if the values mentioned by the customer in the call (e.g. postcode, vehicle type, NCD, claims) match what is shown in the quote. If the customer provided correct details in the call, but the quote shows different values, this may indicate an error made by the call handler.

Step 4: Use the 'Validate Property Attributes' tool to retrieve the property details for this postcode. Check if the returned attributes (e.g. has_garage, has_driveway, property_risk_level) align with what is assumed or declared in the quote. If the quote relies on incorrect property assumptions—for example, assuming a garage where none exists—this could indicate an underwriting error or misclassification. Flag the mismatch and suggest review, then run Step 5 for new quote with corrected parameters. Ensure to include in description the results of the search in comparison to the quote data. If no mismatch skip Step 5 and go to Step 6.

Step 5: Don't use this if the data matches in previous steps or if there is no call transcript. If a mismatch was likely caused by incorrect data entry during the call, note this and suggest review. Use 'Score Quote' tool to create a new price with corrected values.  Otherwise, list all mismatches clearly. Return full description of this process.

Step 6: Summarize above steps in points with details on what tools were used and their results. Ensure to include all details regarding data you have gathered during previous steps. Say if the quote should be approved or rejected. Add a dad joke at the end.
""")




### 💾 Save Agent Output to Table

This cell saves the agent's underwriting decision into the `agent_review` table in Unity Catalog.

- **Steps:**
  1. Creates a single-row DataFrame with `quote_id` and `agent_output`
  2. Registers it as a temporary view (`new_review_data`)
  3. Executes a `MERGE` to upsert the result into `lrcatalog.agentic_underwriting.agent_review`

- **Use case:** Stores decisions and justifications made by the agent for audit, governance, or follow-up review

In [0]:
#save output into a table
from pyspark.sql import Row

# Create a Spark DataFrame with new output
row = Row(quote_id=quote_id, agent_output=agent_output)
new_data = spark.createDataFrame([row])

# Register as temp view for merge
new_data.createOrReplaceTempView("new_review_data")

# Run MERGE to update or insert
spark.sql(f"""
MERGE INTO lrcatalog.agentic_underwriting.agent_review AS target
USING new_review_data AS source
ON target.quote_id = source.quote_id
WHEN MATCHED THEN UPDATE SET target.agent_output = source.agent_output
WHEN NOT MATCHED THEN INSERT (quote_id, agent_output) VALUES (source.quote_id, source.agent_output)
""")

In [0]:
import json

result_summary = {
    "status": "success",
    "quote_id": quote_id,
    "catalog": catalog,
    "schema": schema,
    "table": "underwriting_results"
}

dbutils.notebook.exit(json.dumps(result_summary))